In [36]:
import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

In [37]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

params['dataset'] = 'mnist'
#params['dataset'] = 'mnist'
params['model'] = 'mclr'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'] == 'mnist':
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = True
  params['allow_empty'] = True
  params['evenly'] = False
  params['num_group'] = 5
  params['min_clients'] = 2


"""
Common parameters
"""
params['num_rounds'] = 200
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20
params['seed'] = 233


""" Set the output CSV file name """
info = '{}-{}'.format(params['optimizer'], params['dataset'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  info2 = '-NumG{:1d}-Empty{:.1}-Proximal{:.1}'.format(
    params['num_group'], str(params['allow_empty']), str(params['proximal']))
  if params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-mnist-NumG5-EmptyT-ProximalT.csv


In [38]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])

mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train


1000 Clients in Total


Clustering iteration: 7


In [4]:
server.train()

Training with 20 workers ---
Group 0 is empty.
Group 1 is empty.
Group 2 is empty.
Group 3, clients ['f_00026', 'f_00086', 'f_00002', 'f_00055', 'f_00075', 'f_00093', 'f_00016', 'f_00073', 'f_00054', 'f_00095', 'f_00053', 'f_00092', 'f_00078', 'f_00013', 'f_00007', 'f_00030', 'f_00022', 'f_00024', 'f_00033', 'f_00008']
Group 4 is empty.
Group 0
At round 0 accuracy: 1.0
At round 0 training accuracy: 1.0
At round 0 training loss: 0.22242969274520874
Group 1
At round 0 accuracy: 0.5373665480427047
At round 0 training accuracy: 0.48373015873015873
At round 0 training loss: 1.7255104784454618
Group 2
At round 0 accuracy: 1.0
At round 0 training accuracy: 0.9733333333333334
At round 0 training loss: 0.37313151359558105
Group 3
At round 0 accuracy: 0.033185185185185186
At round 0 training accuracy: 0.032714075007043306
At round 0 training loss: 3.317480685480118
Group 4
At round 0 accuracy: 1.0
At round 0 training accuracy: 0.9953703703703703
At round 0 training loss: 0.19603214169748956
At r

Group 0 is empty.
Group 1, clients ['f_00071']
Group 2 is empty.
Group 3, clients ['f_00037', 'f_00026', 'f_00078', 'f_00091', 'f_00049', 'f_00015', 'f_00093', 'f_00086', 'f_00022', 'f_00013', 'f_00040', 'f_00052', 'f_00012', 'f_00088', 'f_00045', 'f_00066', 'f_00020', 'f_00018']
Group 4, clients ['f_00011']
Group 0
At round 7 accuracy: 0.2727272727272727
At round 7 training accuracy: 0.15217391304347827
At round 7 training loss: 1.55466628074646
Group 1
At round 7 accuracy: 0.18861209964412812
At round 7 training accuracy: 0.17063492063492064
At round 7 training loss: 2.4808860858754507
Group 2
At round 7 accuracy: 0.5294117647058824
At round 7 training accuracy: 0.5466666666666666
At round 7 training loss: 1.7680577039718628
Group 3
At round 7 accuracy: 0.05644444444444444
At round 7 training accuracy: 0.055932119122984375
At round 7 training loss: 2.5606042160067384
Group 4
At round 7 accuracy: 0.7628865979381443
At round 7 training accuracy: 0.7523148148148148
At round 7 training l

KeyboardInterrupt: 

In [24]:
from collections import Counter

group_list = server.group_list
clients = server.clients
g = group_list[3]
c_list = []
for c in clients:
  if c.group == g:
    c_list.append(c)

cnt = Counter()
for c in c_list:
  cnt += Counter(c.eval_data['y'])
print(cnt)
print(sum(cnt.values()))

Counter({3.0: 426, 9.0: 365, 4.0: 268, 8.0: 231, 6.0: 224, 2.0: 221, 7.0: 115, 0.0: 88, 1.0: 81, 5.0: 61})
2080
Error, cluster is empty
Error, cluster is empty
Error, cluster is empty
Clustering iteration: 3


In [41]:
random.seed(10)
cluster = server.clustering_clients(server.clients, n_clusters=5)
for (cm, clients) in cluster.values():
  print(len(clients))

Clustering iteration: 12
186
291
121
195
207
